# Part 1: Data Preprocessing

use Markdown cells to describe what is going on

In [23]:
import requests 
import bs4
import pandas as pd
import re
from math import sin, cos, sqrt, atan2, radians
from io import StringIO
import pandas as pd
import glob
import os

# Part 1: Data Preprocessing

## Uber Dataset

In [ ]:
df_uber=pd.read_csv("uber_rides_sample.csv")

In [ ]:
def distance(row):
    R = 6373.0
    
    lon1 = radians(row["pickup_longitude"])
    lat1 = radians(row["pickup_latitude"])
    
    lon2 = radians(row["dropoff_longitude"])
    lat2 = radians(row["dropoff_latitude"])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
def uber_data_cleaning(df_uber):
    df_uber.dropna()
    df_uber.drop("Unnamed: 0" , axis=1, inplace=True)
    
    pick_long_check=(df_uber["pickup_longitude"]>= -74.242330) & (df_uber["pickup_longitude"]<= -73.717047)
    drop_long_check=(df_uber["dropoff_longitude"]>= -74.242330) & (df_uber["dropoff_longitude"]<= -73.717047)
    pick_latt_check=(df_uber["pickup_latitude"]>= 40.560445) & (df_uber["pickup_latitude"]<= 40.908524)
    drop_latt_check=(df_uber["dropoff_latitude"]>= 40.560445) & (df_uber["dropoff_latitude"]<= 40.908524)

    df_uber=df_uber[pick_long_check & drop_long_check & pick_latt_check & drop_latt_check]
    
    df_uber['pickup_datetime'] = pd.to_datetime(df_uber['pickup_datetime'])
    df_uber['pickup_datetime']=df_uber['pickup_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    df_uber['distance'] = df_uber.apply(distance, axis=1)
    
    return df_uber

In [ ]:
df_uber=uber_data_cleaning(df_uber)

In [ ]:
df_uber

In [ ]:
# df_uber.to_csv('uber_cleaned.csv',index=False)

## Yellow Taxi Dataset

In [ ]:
def link_parser(): 
    # Scrapping out link for each of yellow trip data
    
    response = requests.get('https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page')
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    parse = str(soup.find_all("li")).split()
    pattern = re.compile(r"href=\"https:\/\/s3\.amazonaws\.com\/nyc\-tlc\/trip\+data\/yellow\_tripdata\_20[0-1]\d")
    newlist = list(filter(pattern.match, parse))
    newlist = newlist[42:]
    linklist=[]
    for item in newlist:
        linklist.append(item.split('href="')[1][:-1])
    return linklist


In [ ]:
linklist=link_parser()

In [ ]:
def yellow_taxi_dataset(linklist):
    result=pd.DataFrame()
    #Data collection of 2015's yellow taxi data
    for link in linklist[:12]:
        req = requests.get(link)
        url_content = req.content
    
        s=str(url_content,'utf-8')
        data = StringIO(s) 
        df=pd.read_csv(data, error_bad_lines=False)
        df.drop("improvement_surcharge" , axis=1, inplace=True)
        df = df.iloc[: , :-1]
        df.columns=['vendor_id','pickup_datetime','dropoff_datetime','passenger_count','trip_distance','pickup_longitude',
                'pickup_latitude','rate_code','store_and_fwd_flag','dropoff_longitude','dropoff_latitude','payment_type',
                'fare_amount','surcharge','mta_tax','tip_amount','tolls_amount','total_amount']
        df = df.sample(n=3000)
    
        result=result.append(df)
    #Data collection of years from 2009 to 2014    
    for link in linklist[12:]:
        req = requests.get(link)
        url_content = req.content
    
        s=str(url_content,'utf-8')
        data = StringIO(s) 
        df=pd.read_csv(data, error_bad_lines=False)
        df.columns=['vendor_id','pickup_datetime','dropoff_datetime','passenger_count','trip_distance','pickup_longitude',
                'pickup_latitude','rate_code','store_and_fwd_flag','dropoff_longitude','dropoff_latitude','payment_type',
                'fare_amount','surcharge','mta_tax','tip_amount','tolls_amount','total_amount']
        df = df.sample(n=3000)
    
        result=result.append(df)
        
    return result
    

In [ ]:
df_taxi=yellow_taxi_dataset(linklist)

In [ ]:
df_taxi=pd.read_csv("Yellow_Taxi_Sample.csv")

In [ ]:
def clean_yellowtaxi(df_taxi):
    
    #location filter
    pick_long_check=(df_taxi["pickup_longitude"]>= -74.242330) & (df_taxi["pickup_longitude"]<= -73.717047)
    drop_long_check=(df_taxi["dropoff_longitude"]>= -74.242330) & (df_taxi["dropoff_longitude"]<= -73.717047)
    pick_latt_check=(df_taxi["pickup_latitude"]>= 40.560445) & (df_taxi["pickup_latitude"]<= 40.908524)
    drop_latt_check=(df_taxi["dropoff_latitude"]>= 40.560445) & (df_taxi["dropoff_latitude"]<= 40.908524)

    df_taxi=df_taxi[pick_long_check & drop_long_check & pick_latt_check & drop_latt_check]
    
    #drop columns with too many NAs
    na_bar = len(df_taxi) * .8
    df_taxi = df_taxi.dropna(thresh=na_bar, axis=1)
    
    #add distance column
    df_taxi['distance'] = df_taxi.apply(distance, axis=1)
    
    #drop trip_distance col
    df_taxi = df_taxi.drop(['trip_distance'], axis = 1)

    
    return df_taxi

In [ ]:
#df_taxi=pd.read_csv("Yellow_Taxi_Sample.csv")
df_taxi=clean_yellowtaxi(df_taxi)
df_taxi

In [ ]:
#Test Case for Distance Function
def distance_test():
    
    distance_test=round(distance(df_taxi.iloc[[0]]),2)   
    assert distance_test== 0.88

In [ ]:
distance_test()

## Weather Dataset

In [ ]:
def concact_weather_data():
    # setting the path for joining multiple files
    files = os.path.join("/Users/nat/Desktop/tfa/proj", "*weather.csv") #directory change needed

    # list of merged files returned
    files = glob.glob(files)

    # joining files with concat and store csv
    df_weather = pd.concat(map(pd.read_csv, files), ignore_index=True)
    #df_weather.to_csv('weather_all.csv',index=False)  
    
    return df_weather

In [ ]:
df_weather=concact_weather_data()

In [24]:
df_weather=pd.read_csv("weather_all.csv")

/Users/lisakuai/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,8,9,10,13,17,18,19,40,41,42,61,65,78,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
def clean_weather(df_weather):
    #choose only useful cols
    df_weather_cleaned = df_weather[df_weather.filter(regex='DATE|speed|Speed|Precipitation|precipitation').columns[:8]]
    
    df_weather_hourly = df_weather_cleaned[df_weather_cleaned.filter(regex='DATE|hourly|Hourly').columns[:]]
    df_weather_daily = df_weather_cleaned[df_weather_cleaned.filter(regex='DATE|daily|Daily').columns[:]]
    
    return df_weather_hourly,df_weather_daily

In [26]:
df_weather_hourly,df_weather_daily=clean_weather(df_weather)

# Part 2: Storing Data

#Use SQLAlchemy to create a SQLite
#explanation needs

In [ ]:
#Question:
# 1. Is it correct to store 4 tables into final_proj.db?
# 2. daily weather data; too many missing values
# 3. how to save as schema.sqlp file? we only have .db file now

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import validates

In [ ]:
def database():
    
    engine = create_engine(f"sqlite:///final_project.db")
    Base = declarative_base()
    
    # Convert csv file into sql & insert into data.db
    df_uber.to_sql('Uber_trips', con=engine, index=True, index_label='id', if_exists='replace')
    df_taxi.to_sql('Yellow_Taxi_trips', con=engine, index=True, index_label='id', if_exists='replace')
    df_weather_daily.to_sql('Daily_Weather_Information', con=engine, index=True, index_label='id', if_exists='replace')
    df_weather_hourly.to_sql('Hourly_Weather_Information', con=engine, index=True, index_label='id', if_exists='replace')

In [ ]:
database()

In [27]:
df_weather_hourly

,DATE,HourlyPrecipitation,HourlyWindGustSpeed,HourlyWindSpeed
0,2012-01-01T00:51:00,NaN,NaN,6.0
1,2012-01-01T01:51:00,NaN,NaN,7.0
2,2012-01-01T02:51:00,NaN,NaN,6.0
3,2012-01-01T03:51:00,NaN,NaN,5.0
4,2012-01-01T04:51:00,NaN,NaN,0.0
...,...,...,...,...
77967,2010-12-31T19:51:00,NaN,NaN,5.0
77968,2010-12-31T20:51:00,NaN,NaN,7.0
77969,2010-12-31T21:51:00,NaN,NaN,7.0
77970,2010-12-31T22:51:00,NaN,NaN,6.0


In [31]:
df_weather_daily['DailyAverageWindSpeed'].describe()

count    1246.000000
mean        5.307223
std         2.350127
min         0.700000
25%         3.700000
50%         4.900000
75%         6.500000
max        18.800000
Name: DailyAverageWindSpeed, dtype: float64

# Part 3: Understanding Sata

In [1]:
import sqlite3
connection = sqlite3.connect("final_project.db")
connection

1.	For 01-2009 through 06-2015, what hour of the day was the most popular to take a Yellow Taxi? The result should have 24 bins.

In [2]:
with connection:
    Q1 = connection.execute(
        """SELECT strftime('%H', y.pickup_datetime) as Hour, count(*) as Counts
                                From Yellow_Taxi_trips y
                                where date(y.pickup_datetime)>="2009-01-01" and date(y.pickup_datetime)<="2015-06-30"
                                group by Hour
                                order by Counts desc""")

In [3]:
for row in Q1:
    print(row)

('19', 14246)
('18', 13868)
('20', 13576)
('21', 13047)
('22', 12742)
('14', 11446)
('17', 11335)
('23', 11295)
('12', 11085)
('15', 11070)
('13', 11057)
('09', 10625)
('08', 10471)
('11', 10438)
('10', 10356)
('16', 9615)
('00', 9141)
('07', 8282)
('01', 6766)
('02', 5013)
('06', 4703)
('03', 3685)
('04', 2713)
('05', 2224)


2.	For the same time frame, what day of the week was the most popular to take an Uber? The result should have 7 bins.

In [4]:
with connection:
    Q2 = connection.execute(
        """SELECT strftime('%w', u.pickup_datetime) as weekofday, count(*) as Counts
                                From Uber_trips u
                                where date(u.pickup_datetime)>="2009-01-01" and date(u.pickup_datetime)<="2015-06-30"
                                group by weekofday
                                order by Counts desc""")

In [5]:
for item in Q2:
    print(item)

('5', 30166)
('6', 29599)
('4', 29338)
('3', 28328)
('2', 27526)
('0', 25834)
('1', 24681)


3.	What is the 95% percentile of distance traveled for all hired trips during July 2013?

In [6]:
with connection:
    Q3 = connection.execute(
        """ with newtable as 
                (SELECT distance from Uber_trips
                 Where date(pickup_datetime)<="2013-07-31" and date(pickup_datetime)>="2013-07-1"
            
                 Union all
            
                SELECT distance from Yellow_Taxi_trips
                Where date(pickup_datetime)<="2013-07-31" and date(pickup_datetime)>="2013-07-1")
            
            Select distance as "95% distance"
            FROM newtable
            ORDER BY distance ASC
            LIMIT 1
            OFFSET (SELECT
             COUNT(*)
            FROM newtable
            ) * 95 / 100 - 1
            
            """)

In [7]:
for item in Q3:
    print(item)

(10.430214496267116,)


4.	What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?

In [20]:
with connection:
    Q4 = connection.execute(
        """ with newtable as 
                (SELECT distance, date(pickup_datetime) as date from Uber_trips
                 Where strftime('%Y', pickup_datetime)="2009"
            
                 Union all
            
                SELECT distance, date(pickup_datetime) as date from Yellow_Taxi_trips
                Where strftime('%Y', pickup_datetime)="2009")
            
            Select date, Avg(distance) as Avg_dist 
            FROM newtable
            Group by date
            Order by Count(*) desc
            LIMIT 10
            
            """)

In [21]:
for item in Q4:
    print(item)

('2009-10-23', 2.8750806400754474)
('2009-07-23', 3.141376992097748)
('2009-08-14', 3.1066621910406735)
('2009-12-11', 2.915932452752843)
('2009-06-05', 3.032279559824318)
('2009-02-20', 2.7511630452917912)
('2009-03-06', 2.9318889606681013)
('2009-01-31', 3.1153235637726295)
('2009-12-18', 2.8862065653873272)
('2009-04-18', 3.543311738881416)


5.	Which 10 days in 2014 were the windiest on average, and how many hired trips were made on those days?

In [49]:
with connection:
    Q5 = connection.execute(
        """ with newtable as 
            (Select date, sum(num_trip) as num_trips
                from (SELECT date(pickup_datetime) as date, count(*) as num_trip from Uber_trips
                     Where strftime('%Y', pickup_datetime)="2014"
                     Group by date

                     Union all

                    SELECT date(pickup_datetime) as date, count(*) as num_trip from Yellow_Taxi_trips
                    Where strftime('%Y', pickup_datetime)="2014"
                    Group by date)
                group by date)
                    
               
               
              Select n.date, n.num_trips
              from newtable n
              Join
                (Select date(DATE) as date, DailyAverageWindSpeed
                From Daily_Weather_Information
                Where strftime('%Y', DATE)="2014"
                Order by DailyAverageWindSpeed Desc
                Limit 10) w
            on n.date=w.date
                
            
          

                """)

In [50]:
for item in Q5:
    print(item)

('2014-03-13', 206)
('2014-01-07', 185)
('2014-02-13', 124)
('2014-01-02', 139)
('2014-03-26', 206)
('2014-12-07', 171)
('2014-12-08', 182)
('2014-03-29', 229)
('2014-11-02', 182)
('2014-01-03', 98)


6.	During Hurricane Sandy in NYC (Oct 29-30, 2012), plus the week leading up and the week after, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed? There should be an entry for every single hour, even if no rides were taken, no precipitation was measured, or there was no wind.